<a href="https://colab.research.google.com/github/nakamae11/-/blob/main/%E6%A5%BD%E5%A4%A9API%E9%80%A3%E6%90%BA%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%82%AB%E3%82%B9%E3%82%BF%E3%83%9E%E3%82%A4%E3%82%BA%E5%8F%AF%E8%83%BDVer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 必要ライブラリのインストール（初回のみ）
!pip install --quiet requests gspread oauth2client

import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files

# 楽天API設定
APP_ID = "1016526773867927317"
KEYWORD = "コンセント収納"

# スプレッドシート名
SPREADSHEET_NAME = "楽天商品リスト"

# Google認証情報アップロードを促す
print("認証情報JSONファイルをアップロードしてください。")
uploaded = files.upload()

# 先ほどアップロードしたファイル名を取得
credentials_file = list(uploaded.keys())[0]

# 楽天APIから商品取得
url = "https://app.rakuten.co.jp/services/api/IchibaItem/Search/20170706"
params = {
    "applicationId": APP_ID,
    "keyword": KEYWORD,
    "sort": "-reviewCount",
    "hits": 30,
    "format": "json"
}

response = requests.get(url, params=params)
items = response.json()["Items"]

# Google Sheets認証
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
client = gspread.authorize(creds)

# スプレッドシート取得
sheet = client.open(SPREADSHEET_NAME).sheet1

# ヘッダー書き込み
sheet.update('A1', [["商品名", "価格", "レビュー数", "商品URL"]])

# 商品データを書き込み
rows = []
for item in items:
    i = item["Item"]
    rows.append([i["itemName"], i["itemPrice"], i["reviewCount"], i["itemUrl"]])

sheet.update(f"A2:D{len(rows)+1}", rows)

print("✅ スプレッドシートに出力完了！")


認証情報JSONファイルをアップロードしてください。


Saving reviewcountsearch-1034866f6360.json to reviewcountsearch-1034866f6360 (3).json


<ipython-input-1-cd440a493d71>:45: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [["商品名", "価格", "レビュー数", "商品URL"]])
<ipython-input-1-cd440a493d71>:53: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f"A2:D{len(rows)+1}", rows)


✅ スプレッドシートに出力完了！


In [3]:
# ライブラリのインストール（最初だけ）
!pip install -q gspread oauth2client requests ipywidgets

# --------- ① 各種インポート ---------
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import ipywidgets as widgets
from IPython.display import display

# --------- ② Googleスプレッドシート連携 ---------
# 認証（アップロードした credentials.json を使う）
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('reviewcountsearch-1034866f6360.json', scope)
client = gspread.authorize(creds)

# スプレッドシート名
SPREADSHEET_NAME = '楽天商品リスト'
sheet = client.open(SPREADSHEET_NAME).sheet1

# --------- ③ ウィジェット（フォームUI） ---------
keyword_widget = widgets.Text(value='コーヒー', description='キーワード:')
sort_widget = widgets.Dropdown(
    options=[
        ('標準', 'standard'),
        ('レビューが多い順', '-reviewCount'),
        ('価格が安い順', '+itemPrice'),
        ('価格が高い順', '-itemPrice'),
        ('新着順', '-updateTimestamp')
    ],
    value='-reviewCount',
    description='ソート:'
)
hits_widget = widgets.BoundedIntText(value=30, min=1, max=100, description='件数:')

display(keyword_widget, sort_widget, hits_widget)

# --------- ④ 実行ボタン ---------
button = widgets.Button(description="検索してシート出力")

def on_button_clicked(b):
    keyword = keyword_widget.value
    sort = sort_widget.value
    hits = hits_widget.value
    app_id = "1016526773867927317"  # 楽天APIのアプリID

    # API呼び出し
    url = "https://app.rakuten.co.jp/services/api/IchibaItem/Search/20220601"
    params = {
        "applicationId": app_id,
        "keyword": keyword,
        "sort": sort,
        "hits": hits,
        "format": "json"
    }
    res = requests.get(url, params=params).json()

    # スプレッドシートの先頭にヘッダー
    sheet.clear()
    sheet.append_row(["商品名", "価格", "レビュー数", "商品URL", "ショップ名", "ショップURL"])

    # 結果をスプレッドシートに書き込み
    for item in res.get("Items", []):
        data = item["Item"]
        row = row = [
                        data["itemName"],
                        data["itemPrice"],
                        data["reviewCount"],
                        data["itemUrl"],
                        data["shopName"],
                        data["shopUrl"]
                    ]
        sheet.append_row(row)

    print(f"{len(res['Items'])} 件をスプレッドシートに出力しました！")

button.on_click(on_button_clicked)
display(button)


Text(value='コーヒー', description='キーワード:')

Dropdown(description='ソート:', index=1, options=(('標準', 'standard'), ('レビューが多い順', '-reviewCount'), ('価格が安い順', '+…

BoundedIntText(value=30, description='件数:', min=1)

Button(description='検索してシート出力', style=ButtonStyle())

30 件をスプレッドシートに出力しました！
